In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于在线预测的AutoML文本实体提取模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_text_entity_extraction_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_text_entity_extraction_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库创建文本实体提取模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行在线预测。

### 数据集

本教程使用的数据集是来自[国家生物技术信息中心](https://www.ncbi.nlm.nih.gov/)的[NCBI疾病研究摘要数据集](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/)。您在本教程中将使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本中创建一个AutoML文本实体提取模型，并部署到在线预测。您还可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个serving `Endpoint`资源。
- 进行预测。
- 撤销`Model`的部署。

### 成本

本教程使用 Google Cloud（GCP） 的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的GA版本* google-cloud-storage *库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

重新启动内核

安装了Vertex客户端库和Google *cloud-storage*后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

*如果您有这个选项，请确保在GPU运行时下运行此笔记本。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex API和Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK为此笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量用于本笔记本的其余操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不使用多区域存储桶进行Vertex的训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行现场教程，您可能会使用共享的测试账户或项目。为了避免用户在创建资源时发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐户

**如果您正在使用Google Cloud Notebook**，您的环境已经通过身份验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入"Vertex"，然后选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，然后选择**Storage Object Admin**。

点击创建。包含您的密钥的JSON文件将下载到本地环境。

在下面的单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设定变量

接下来，设定一些在教程中使用的变量。
### 导入库并定义常量

导入顶点客户端库

将顶点客户端库导入我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

### 顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、流水线和端点服务终端点。
- `PARENT`：顶点位置根路径，用于数据集、模型、作业、流水线和端点资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML常量

设置专用于AutoML数据集和训练的常量：

- 数据集模式：告诉`数据集`资源服务它是什么类型的数据集。
- 数据标记（注释）模式：告诉`数据集`资源服务数据如何标记（注释）。
- 数据集训练模式：告诉`管道`资源服务为哪种任务（例如分类）训练模型。

In [ ]:
# Text Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/text_1.0.0.yaml"
# Text Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/text_extraction_io_format_1.0.0.yaml"
# Text Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_text_extraction_1.0.0.yaml"

# 教程

现在，您已经准备好开始创建自己的AutoML文本实体提取模型。

## 设置客户端

Vertex 客户端库采用客户端/服务器模型。在您的一侧（Python 脚本）中，您将创建一个客户端，该客户端将向 Vertex 服务器发送请求并接收响应。

在本教程中，您将针对工作流中的不同步骤使用不同的客户端。因此，请提前设置它们全部。

- 数据集服务用于 `数据集` 资源。
- 模型服务用于 `模型` 资源。
- 训练管道服务。
- 部署端点服务。
- 服务预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

数据集

现在您的客户准备好了，训练模型的第一步是创建一个托管的数据集实例，然后将标记的数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），具有以下参数：
   - `display_name`：您选择的用于显示的人类可读名称。
   - `metadata_schema_uri`：数据集类型的模式。
3. 调用客户端数据集服务方法`create_dataset`，具有以下参数：
   - `parent`：`Database`、`Model`和`Endpoint`资源的Vertex位置根路径。
   - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

`operation`对象是Vertex处理长时间运行操作的异步调用的方式。虽然这一步通常很快，但在项目中第一次使用时，由于配置需求，可能会有更长的延迟。

您可以使用`operation`对象来获取操作（例如，创建`Dataset`资源）的状态，或者通过调用操作方法来取消操作：

| 方法       | 描述          |
| ----------- | ----------- |
| result()     | 等待操作完成并返回JSON格式的结果对象。       |
| running()   | 返回操作是否仍在运行的True/False。       |
| done()      | 返回操作是否已完成的True/False。 |
| canceled()  | 返回操作是否已取消的True/False。 |
| cancel()     | 取消操作（可能需要最多30秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("biomedical-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

Vertex中用于文本的`Dataset`资源对于您的文本实体提取数据有一些要求。

- 文本示例必须存储在JSONL文件中。与文本分类和情感分析不同，不支持CSV索引文件。
- 示例必须是内联文本或存储在Cloud Storage存储桶中的参考文本文件。

#### JSONL

对于文本实体提取，JSONL 文件有一些要求：

- 每个数据项都是一个单独的 JSON 对象，位于单独的一行上。
- 键/值对 `text_segment_annotations` 是每个实体的在文本中的字符起始/结束位置与对应标签的列表。
 - `display_name`：标签。
 - `start_offset/end_offset`：实体起始/结束的字符偏移量。
- 键/值对 `text_content` 是文本内容。

    {'text_segment_annotations': [{'end_offset': value, 'start_offset': value, 'display_name': label}, ...], 'text_content': text}

*注意*：字典键字段也可以使用驼峰命名法。例如，'display_name' 也可以写为 'displayName'。

#### 云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中JSONL索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://ucaip-test-us-central1/dataset/ucaip_ten_dataset.jsonl"

快速查看您的数据

您将使用存储在公共云存储桶中的 NCBI生物医学数据集的一个版本，使用一个 JSONL 索引文件。

首先快速查看数据。通过计算 JSONL 索引文件中对象的数量（`wc -l`），然后查看前几行来统计示例数量。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex Dataset 资源中。使用这个辅助函数 `import_data` 来导入数据。该函数执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，并使用以下参数：
 - `name`：您为 `Dataset` 资源分配的人类可读的名称（例如，生物医学）。
 - `import_configs`：导入配置。

- `import_configs`：一个包含字典的 Python 列表，其中包含键/值条目：
 - `gcs_sources`：指向一个或多个索引文件路径的 URI 列表。
 - `import_schema_uri`：用于标识标签类型的模式。

`import_data()` 方法返回一个长时间运行的 `operation` 对象。这将需要几分钟才能完成。如果您在一次实时教程中，这会是一个很好的时间来提问或休息一下。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

训练模型

现在使用您的 Vertex `Dataset` 资源训练一个 AutoML 文本实体提取模型。要训练模型，请执行以下步骤：

1. 为 `Dataset` 资源创建一个 Vertex 训练管道。
2. 执行管道以开始训练。

### 创建训练管道

您可能会问，我们用管道做什么？通常在工作（如训练）具有多个步骤时，您会使用管道，通常按顺序进行：执行步骤A，执行步骤B等等。通过将步骤放入管道中，我们获得以下好处：

1. 可以用于后续训练工作的重复使用。
2. 可以作为批处理作业进行容器化并运行。
3. 可以分布式运行。
4. 所有步骤都与同一管道作业相关联，以跟踪进度。

使用此辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：管道作业的可读名称。
- `model_name`：模型的可读名称。
- `dataset`：Vertex完全限定的数据集标识符。
- `schema`：数据集标注（注释）训练架构。
- `task`：描述训练作业要求的字典。

辅助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，它接受以下参数：

- `parent`：您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练作业的完整规格。

现在让我们更深入地了解构造 `training_pipeline` 规格的 *最小* 要求：

- `display_name`：管道作业的可读名称。
- `training_task_definition`：数据集标注（注释）训练架构。
- `training_task_inputs`：描述训练作业要求的字典。
- `model_to_upload`：模型的可读名称。
- `input_data_config`：数据集规格。
 - `dataset_id`：仅为 Vertex 数据集标识符（非完全限定）--这是完全限定标识符的最后部分。
 - `fraction_split`：如果指定，用于训练、测试和验证的数据集百分比。否则，百分比将由 AutoML 自动选取。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他参数不同，`task`字段需要一个Google protobuf结构，而不是Python（类似JSON）字典。使用`json_format.ParseDict`方法进行转换。

您需要指定的最少字段是：

- `multi_label`：指定此是否为多标签（而不是单一）分类。
- `budget_milli_node_hours`：模型训练的最大计费预算时间，其中 1000 = 1小时。
- `model_type`：部署模型的类型：
  - `CLOUD`：用于部署到Google Cloud。
- `disable_early_stopping`：指定是否为True/False，以让AutoML自行判断是否提前停止训练或在整个预算时间内训练。

最后，通过调用助手函数`create_pipeline`来创建流水线，它会返回一个训练流水线对象的实例。

In [ ]:
PIPE_NAME = "biomedical_pipe-" + TIMESTAMP
MODEL_NAME = "biomedical_model-" + TIMESTAMP

task = json_format.ParseDict(
    {
        "multi_label": False,
        "budget_milli_node_hours": 8000,
        "model_type": "CLOUD",
        "disable_early_stopping": False,
    },
    Value(),
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练流水线的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取培训流水线的信息

现在，只需获取此培训流水线实例的流水线信息。通过调用作业客户端服务的`get_training_pipeline`方法，辅助函数可以获取此作业的作业信息，参数如下：

- `name`：Vertex完全合格的流水线标识符。

当模型训练完成时，流水线状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

训练以上模型可能需要长达120分钟的时间。

一旦您的模型训练完成，您可以通过将 `end_time` 减去 `start_time` 来计算训练模型所需的实际时间。对于您的模型，您需要知道完全合格的 Vertex 模型资源标识符，即管道服务分配给它的。您可以从返回的管道实例中获取此信息，作为字段 `model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在您的模型已经训练好，您可以获取关于您的模型的一些信息。

评估模型资源

现在找出模型服务认为你的模型有多好。在训练过程中，将数据集的一部分设置为测试（保留）数据，这些数据由管道服务用来评估模型。

### 列出所有切片的评估

使用这个辅助函数`list_model_evaluations`，它接受以下参数：

- `name`：`Model`资源的Vertex完全限定模型标识符。

这个辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估 -- 你可能只有一个，然后我们打印出每个评估指标中的所有关键名称，对于一个小的集合（`confusionMatrix`和`confidenceMetrics`），您将打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("confusionMatrix", metrics["confusionMatrix"])
        print("confidenceMetrics", metrics["confidenceMetrics"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

部署“Model”资源

现在部署使用AutoML创建的经过训练的Vertex“Model”资源。这需要两个步骤：

1. 为部署“Model”资源创建一个“Endpoint”资源。

2. 将“Model”资源部署到“Endpoint”资源。

### 创建一个`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个用于部署模型以进行预测服务的终端节点，带有以下参数：

- `display_name`：用于`Endpoint`资源的可读性高的名称。

辅助函数使用终端节点客户端服务的`create_endpoint`方法，该方法接受以下参数：

- `display_name`：用于`Endpoint`资源的可读性高的名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为为了进行服务可能需要一些时间来准备`Endpoint`资源。您可以调用`response.result()`，这是一个同步调用，会在Endpoint资源准备就绪时返回。这个辅助函数返回了`Endpoint`资源的Vertex完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "biomedical_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获得您创建的“Endpoint”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

在处理您的在线预测请求时，您有多种选择来扩展计算实例：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为一。

- 手动扩展：在线预测请求分布在您手动指定的固定数量的计算实例中。
  - 将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为相同的节点数。当模型首次部署到实例时，固定数量的计算实例被配置并且在线预测请求均匀分布在它们之间。

- 自动扩展：在线预测请求分布在可扩展数量的计算实例中。
  - 将计算实例的最小 (`MIN_NODES`) 数量设置为模型首次部署时要配置和取消配置的数量，并根据负载情况设置最大 (`MAX_NODES`) 计算实例的数量。

计算实例的最小数量对应于字段 `min_replica_count`，最大数量对应于字段 `max_replica_count`，在您随后的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `模型` 资源到 `终端点` 资源

使用此辅助函数 `deploy_model` 将 `模型` 资源部署到为提供预测创建的 `终端点` 资源，使用以下参数：

- `model`: 要从训练管道中上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的可读名称。
- `endpoint`: 要将模型部署到的 Vertex 完全限定终端点标识符。

辅助函数调用 `终端点` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`: 要将 `模型` 资源部署到的 Vertex 完全限定 `终端点` 资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 流量百分比到达终端点，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则可指定为 **{ "0": 100 }**，其中 "0" 指的是上传的模型，100 表示百分之百的流量。
   - 如果在终端点上有现有模型，将对流量进行分离，则可以使用 `model_id` 指定为 **{ "0": percent, model_id: percent, ... }**，其中 `model_id` 是要部署到的终端点的现有模型的模型标识符。这些百分比必须总和为 100。

现在让我们更深入地了解 `deployed_model` 参数。该参数被指定为一个带有最小所需字段的 Python 字典：

- `model`: 要部署的（上传）模型的 Vertex 完全限定模型标识符。
- `display_name`: 部署模型的可读名称。
- `disable_container_logging`: 这将禁止容器事件（例如执行失败）的日志记录（默认情况下启用容器日志记录）。通常在进行部署调试时启用容器日志记录，然后在生产环境部署时禁用。
- `automatic_resources`: 这指的是多少个冗余计算实例（副本）。在这个例子中，我们将其设置为一个（不进行复制）。

### 流量分流

现在让我们深入了解 `traffic_split` 参数。该参数被指定为一个 Python 字典。起初可能有点混淆。我来解释一下，您可以将模型的多个实例部署到一个终端点，然后设置每个实例的流量百分比。

为什么这样做？也许您已经在生产中部署了先前版本 -- 让我们称其为 v1。您对 v2 的模型评估更好，但在部署到生产环境之前，您并不确定它是否真的更好。因此，在流量分流的情况下，您可能希望将 v2 部署到与 v1 相同的终端点，但只分配 10% 的流量。这样，您可以在不干扰大多数用户的情况下监视其性能，直到做出最终决定。

### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，该操作将一直阻塞直到模型被部署。如果这是第一次将模型部署到终端点，可能需要额外几分钟来完成资源的设置。

In [ ]:
DEPLOYED_NAME = "biomedical_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "automatic_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

进行在线预测请求

现在对部署的模型进行在线预测。

制作测试项目

您将使用合成数据作为一个测试数据项目。不要担心我们在使用合成数据 - 我们只是想展示如何进行预测。

In [ ]:
test_item = 'Molecular basis of hexosaminidase A deficiency and pseudodeficiency in the Berks County Pennsylvania Dutch.\tFollowing the birth of two infants with Tay-Sachs disease ( TSD ) , a non-Jewish , Pennsylvania Dutch kindred was screened for TSD carriers using the biochemical assay . A high frequency of individuals who appeared to be TSD heterozygotes was detected ( Kelly et al . , 1975 ) . Clinical and biochemical evidence suggested that the increased carrier frequency was due to at least two altered alleles for the hexosaminidase A alpha-subunit . We now report two mutant alleles in this Pennsylvania Dutch kindred , and one polymorphism . One allele , reported originally in a French TSD patient ( Akli et al . , 1991 ) , is a GT-- > AT transition at the donor splice-site of intron 9 . The second , a C-- > T transition at nucleotide 739 ( Arg247Trp ) , has been shown by Triggs-Raine et al . ( 1992 ) to be a clinically benign " pseudodeficient " allele associated with reduced enzyme activity against artificial substrate . Finally , a polymorphism [ G-- > A ( 759 ) ] , which leaves valine at codon 253 unchanged , is described'

### 进行预测

现在你有一个测试项。使用这个辅助函数`predict_item`，它接受以下参数：

- `filename`：测试项的云存储路径。
- `endpoint`：`Endpoint`资源的 Vertex 完全限定标识符，其中部署了`Model`资源。
- `parameters_dict`：用于提供预测结果的额外过滤参数。

此函数使用以下参数调用预测客户端服务的`predict`方法：

- `endpoint`：`Model`资源部署的`Endpoint`资源的 Vertex 完全限定标识符。
- `instances`：要预测的实例（文本文件）列表。
- `parameters`：用于提供预测结果的额外过滤参数。*注意*，文本模型不支持额外参数。

#### 请求

每个实例的格式为：

    { 'content': text_item }

由于`predict()`方法可以接受多个项（实例），你将你的单个测试项发送为一个测试项的列表。最后一步，你将实例列表封装为谷歌的 protobuf 格式 -- 这是我们传递给`predict()`方法的内容。

#### 响应

`response`对象返回一个列表，列表中的每个元素对应请求中的相应数据项。你将看到每个预测的输出 -- 在我们的例子中只有一个：

- `prediction`：分配给从文本中提取的每个实体的 ID 列表。
- `confidences`：每个实体的置信水平（0 到 1 之间）。
- `display_names`：每个实体的标签名称。
- `textSegmentStartOffsets`：文本中实体的字符起始位置。
- `textSegmentEndOffsets`：文本中实体的字符结束位置。

In [ ]:
def predict_item(data, endpoint, parameters_dict):

    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{"content": data}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", dict(prediction))
    return response


response = predict_item(test_item, endpoint_id, None)

取消部署 `Model` 资源

现在从服务的 `Endpoint` 资源中取消部署您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：在将 `Model` 资源部署到时由终端服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。

这个函数调用终端客户端服务的 `undeploy_model` 方法，带有以下参数：

- `deployed_model_id`：在将 `Model` 资源部署时由终端服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在 `Endpoint` 资源的其余部署模型之间分配流量。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以通过将其设置为空对象 {} 来简单地将 `traffic_split` 留空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理本项目中使用的所有GCP资源，您可以[删除您用于教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME